In [17]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# figure out the correct path
machop_path = Path(".").resolve().parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

from chop.tools.checkpoint_load import load_model
from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes.graph import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.ir.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")


INFO     Set logging level to info


In [26]:
batch_size = 256
model_name = "jsc-tiny"
dataset_name = "jsc"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
    # custom_dataset_cache_path="../../chop/dataset"
)
data_module.prepare_data()
data_module.setup()

model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False,
    checkpoint = None)

# LAB1_CUSTOM_PATH = "/home/bkt123/dev/advanced-deep-learning-systems/mase/mase_output/lab-1_jsc-custom/software/training_ckpts/best.ckpt"
# model = load_model(load_name=LAB1_CUSTOM_PATH, load_type="pl", model=model)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
    max_batches=1
)

dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

In [27]:
from chop.actions import train
import torch

# print(isinstance(mg.model, torch.nn.Module))

model = mg.model
model_info = get_model_info('jsc-tiny')
dataset_info = get_dataset_info('jsc')
task = "cls"

train_params = {
    "model": model,
    "model_info": model_info,
    "data_module": data_module,
    "dataset_info": dataset_info,
    "task": task,
    "optimizer": "adam",
    "learning_rate": 1e-3,
    "weight_decay": 0,
    "plt_trainer_args": {
        "max_epochs": 1,
    }, 
    "auto_requeue": False,
    "save_path": None,
    "visualizer": None,
    "load_name": None,
    "load_type": None
}

train(**train_params)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs



  | Name      | Type               | Params
-------------------------------------------------
0 | model     | GraphModule        | 85    
1 | loss_fn   | CrossEntropyLoss   | 0     
2 | acc_train | MulticlassAccuracy | 0     
3 | acc_val   | MulticlassAccuracy | 0     
4 | acc_test  | MulticlassAccuracy | 0     
5 | loss_val  | MeanMetric         | 0     
6 | loss_test | MeanMetric         | 0     
-------------------------------------------------
85        Trainable params
0         Non-trainable params
85        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/bkt123/anaconda3/envs/mase/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.
/home/bkt123/anaconda3/envs/mase/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [29]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    if get_mase_op(node) == 'linear':
        print(node.name)
        print(50*'-')
        # print(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # print(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # print(node.meta['mase'].parameters['common']['args']['weight']['value'])
        # print(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])
        pprint(mg.modules[node.target].weight)
        print(50*'-')

seq_blocks_0
--------------------------------------------------
Parameter containing:
tensor([[-0.0305, -0.1776, -0.8128,  0.7068, -1.0469, -0.0561,  0.2084,  0.1563,
         -0.1131,  0.0229,  0.3239, -0.0969,  0.1206,  0.0612,  0.4346,  1.3277],
        [ 0.7038,  0.0915, -0.5083, -0.5458, -0.0991, -0.0290, -0.1112, -0.0947,
          0.1561,  0.2029,  0.4555, -0.3466,  0.3300, -0.3184, -0.6688, -0.7392],
        [ 0.1082,  0.3417,  0.2082, -1.2965,  0.4872, -0.5560, -0.0635, -0.4345,
          0.0452, -0.2312,  0.2072, -0.3086,  0.3945, -0.1742,  0.1094, -1.2876],
        [-0.0452, -0.0084,  0.5183, -0.8814,  0.2036, -0.9909, -0.2197, -0.1152,
          0.2549,  0.0219, -0.0409,  0.0456,  0.1760, -0.2004,  0.8417, -1.1798],
        [-0.1375,  0.2932,  0.6886,  0.2977,  0.3908,  0.3663, -0.2447,  0.2614,
         -0.4381, -0.3118, -0.3304,  0.3036, -0.3149,  0.2821, -0.1607,  0.0280]],
       requires_grad=True)
--------------------------------------------------


In [30]:
from chop.passes.graph.transforms import (
    prune_transform_pass,
   
)
pass_args = {
    "weight":{
        "scope" : "global",
        "granularity" : "elementwise",
        "method" :  "l1-norm",
        "sparsity" : 0.5,},
    "activation":{
        "scope" : "global",
        "granularity" : "elementwise",
        "method" : "l1-norm",
        "sparsity" : 0.5,
    },
}
 
mg, _ = prune_transform_pass(mg, pass_args)


In [31]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    if get_mase_op(node) == 'linear':
        print(node.name)
        print(50*'-')
        # pprint(node.meta['mase'].parameters['common'])
        # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
        pprint(mg.modules[node.target].weight)
        pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
        # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])
        print(50*'-')


seq_blocks_0
--------------------------------------------------
tensor([[-0.0000, -0.0000, -0.8128,  0.7068, -1.0469, -0.0000,  0.0000,  0.0000,
         -0.0000,  0.0000,  0.3239, -0.0000,  0.0000,  0.0000,  0.4346,  1.3277],
        [ 0.7038,  0.0000, -0.5083, -0.5458, -0.0000, -0.0000, -0.0000, -0.0000,
          0.0000,  0.0000,  0.4555, -0.3466,  0.3300, -0.3184, -0.6688, -0.7392],
        [ 0.0000,  0.3417,  0.0000, -1.2965,  0.4872, -0.5560, -0.0000, -0.4345,
          0.0000, -0.0000,  0.0000, -0.3086,  0.3945, -0.0000,  0.0000, -1.2876],
        [-0.0000, -0.0000,  0.5183, -0.8814,  0.0000, -0.9909, -0.0000, -0.0000,
          0.0000,  0.0000, -0.0000,  0.0000,  0.0000, -0.0000,  0.8417, -1.1798],
        [-0.0000,  0.2932,  0.6886,  0.2977,  0.3908,  0.3663, -0.0000,  0.2614,
         -0.4381, -0.3118, -0.3304,  0.3036, -0.3149,  0.2821, -0.0000,  0.0000]],
       grad_fn=<MulBackward0>)
tensor([[False, False,  True,  True,  True, False, False, False, False, False,
          

In [32]:
from chop.actions import train
import torch

# print(isinstance(mg.model, torch.nn.Module))

model = mg.model
model_info = get_model_info('jsc-tiny')
dataset_info = get_dataset_info('jsc')
task = "cls"

train_params = {
    "model": model,
    "model_info": model_info,
    "data_module": data_module,
    "dataset_info": dataset_info,
    "task": task,
    "optimizer": "adam",
    "learning_rate": 1e-3,
    "weight_decay": 0,
    "plt_trainer_args": {
        "max_epochs": 1,
    }, 
    "auto_requeue": False,
    "save_path": None,
    "visualizer": None,
    "load_name": None,
    "load_type": None
}

train(**train_params)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs



  | Name      | Type               | Params
-------------------------------------------------
0 | model     | GraphModule        | 85    
1 | loss_fn   | CrossEntropyLoss   | 0     
2 | acc_train | MulticlassAccuracy | 0     
3 | acc_val   | MulticlassAccuracy | 0     
4 | acc_test  | MulticlassAccuracy | 0     
5 | loss_val  | MeanMetric         | 0     
6 | loss_test | MeanMetric         | 0     
-------------------------------------------------
85        Trainable params
0         Non-trainable params
85        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/bkt123/anaconda3/envs/mase/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.
/home/bkt123/anaconda3/envs/mase/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [33]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op


# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    if get_mase_op(node) == 'linear':
        print(node.name)
        print(50*'-')
        # pprint(node.meta['mase'].parameters['common'])
        # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
        pprint(mg.modules[node.target].weight)
        pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
        # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])

        print(50*'-')

seq_blocks_0
--------------------------------------------------
tensor([[-0.0000, -0.0000, -1.6369,  0.5552, -0.7419, -0.0000,  0.0000,  0.0000,
         -0.0000,  0.0000,  0.7397, -0.0000,  0.0000,  0.0000,  0.7932,  1.3671],
        [ 0.8332,  0.0000, -0.4090, -1.0126, -0.0000, -0.0000, -0.0000, -0.0000,
          0.0000,  0.0000,  0.6031, -0.1658,  0.4106, -0.2976, -0.7187, -1.2277],
        [ 0.0000,  0.0189,  0.0000, -0.1738,  0.2096, -1.7474, -0.0000, -0.3942,
          0.0000, -0.0000,  0.0000, -0.0905,  0.3919, -0.0000,  0.0000, -1.5387],
        [-0.0000, -0.0000,  0.2736, -0.5186,  0.0000, -1.3009, -0.0000, -0.0000,
          0.0000,  0.0000, -0.0000,  0.0000,  0.0000, -0.0000,  0.2842, -1.3053],
        [-0.0000,  0.1588,  1.3117, -0.0839,  0.2535,  0.0833, -0.0000,  0.2678,
         -0.4250, -0.1340, -0.2888,  0.2900, -0.3746,  0.2911, -0.0000,  0.0000]],
       grad_fn=<MulBackward0>)
tensor([[False, False,  True,  True,  True, False, False, False, False, False,
          